# Analysis

**Initializing**

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.simplefilter(action='ignore', category=pd.errors.SettingWithCopyWarning)

from Brands import Brands
from Analyzer import Analyzer
from Solver import Solver

# Set the aesthetic style of the plots
sns.set(style="whitegrid")

#an = Analyzer(marketing_mix_segment_weights=move_mkt_weights, last_period=2)
an = Analyzer(last_period=6, sector="Vodites")

son = Brands(sector="Vodites")

solver = Solver(sector="Vodites")


d:\Uni\Anno III\UCSD\112\Markstrat\Code\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Vodites
./Exports/TeamExport_A46051_Alpha_M_Period 6.xlsx loaded
No weights provided
Vodites
./Exports/TeamExport_A46051_Alpha_M_Period 6.xlsx loaded
Attributes file:./Attributes/Vodites/attributes_Vodites_6.json


**Percentage expenditure by segment for each product**

In [2]:
df_marketing_mixes = son.get_marketing_mixes(capped=False)
df_marketing_mixes


,Innovators,Early Adopters,Followers
0,0.148718,0.148718,0.702564
1,0.324706,0.378824,0.296471


## Semantic scales

In [14]:
son.df_segments_semantic

,Segment,Period,Resolution,Energy Efficiency,Carbon Footprint,Connectivity,No. of Apps,Price
Index,,,,,,,,
Adopters_6,Adopters,6,4.88,4.23,4.76,4.47,4.43,5.26
Followers_6,Followers,6,4.13,4.72,2.95,3.77,3.65,4.53
Innovators_6,Innovators,6,5.31,3.57,4.14,5.13,3.83,5.84
Early Adopters_5,Adopters,5,4.73,4.15,4.82,4.25,4.39,5.32
Followers_5,Followers,5,3.86,4.69,3.14,3.67,3.52,4.59
Innovators_5,Innovators,5,5.23,3.45,4.21,5.03,3.78,6.05
Early Adopters_4,Adopters,4,4.54,4.05,4.88,3.97,4.34,5.43
Followers_4,Followers,4,3.45,4.53,3.35,3.29,3.33,4.7
Innovators_4,Innovators,4,5.14,3.33,4.28,4.93,3.74,6.15


In [4]:
son.df_brands_semantic

,Resolution,Energy Efficiency,Carbon Footprint,Connectivity,No. of Apps,Price
MARKET : Vodites,,,,,,
MEMIMMO,2.69,5.39,4.41,2.60,3.10,4.86
REEGO,5.24,2.81,4.41,4.91,3.64,5.82


### Compute Distances between Segments and Products 

#### Compute closest brands for each segment

In [5]:
features_name = ["Resolution", "Energy Efficiency", "Carbon Footprint", "Connectivity", "No. of Apps", "Price"]

In [6]:
son.df_brands_semantic

,Resolution,Energy Efficiency,Carbon Footprint,Connectivity,No. of Apps,Price
MARKET : Vodites,,,,,,
MEMIMMO,2.69,5.39,4.41,2.60,3.10,4.86
REEGO,5.24,2.81,4.41,4.91,3.64,5.82


In [7]:
df_seg_sem = son.df_segments_semantic[:5][features_name]
df_seg_sem


_ = an.get_n_closest(df_base=df_seg_sem, df_performers=son.df_brands_semantic, num_top=2, max_distance_1D=6, distance_metric=0)

({'Adopters_6': {'MEMIMMO': nan, 'REEGO': nan}, 'Followers_6': {'MEMIMMO': nan, 'REEGO': nan}, 'Innovators_6': {'MEMIMMO': nan, 'REEGO': nan}, 'Early Adopters_5': {'MEMIMMO': nan, 'REEGO': nan}, 'Followers_5': {'MEMIMMO': nan, 'REEGO': nan}}, {'Adopters_6': {'MEMIMMO': array([       nan,        nan,        nan,        nan,        nan,
       0.07604563]), 'REEGO': array([       nan,        nan,        nan,        nan,        nan,
       0.10646388])}, 'Followers_6': {'MEMIMMO': array([       nan,        nan,        nan,        nan,        nan,
       0.07284768]), 'REEGO': array([       nan,        nan,        nan,        nan,        nan,
       0.28476821])}, 'Innovators_6': {'MEMIMMO': array([       nan,        nan,        nan,        nan,        nan,
       0.16780822]), 'REEGO': array([       nan,        nan,        nan,        nan,        nan,
       0.00342466])}, 'Early Adopters_5': {'MEMIMMO': array([       nan,        nan,        nan,        nan,        nan,
       0.08646617]

#### Compute closest Segments for each Brand

In [8]:
_ = an.get_n_closest(df_base=son.df_brands_semantic, df_performers=df_seg_sem, max_distance_1D=6)

({'MEMIMMO': {'Adopters_6': nan, 'Followers_6': nan, 'Innovators_6': nan, 'Early Adopters_5': nan, 'Followers_5': nan}, 'REEGO': {'Adopters_6': nan, 'Followers_6': nan, 'Innovators_6': nan, 'Early Adopters_5': nan, 'Followers_5': nan}}, {'MEMIMMO': {'Adopters_6': array([nan, nan, nan, nan, nan, 0.08230452674897108], dtype=object), 'Followers_6': array([nan, nan, nan, nan, nan, 0.06790123456790124], dtype=object), 'Innovators_6': array([nan, nan, nan, nan, nan, 0.2016460905349793], dtype=object), 'Early Adopters_5': array([nan, nan, nan, nan, nan, 0.09465020576131686], dtype=object), 'Followers_5': array([nan, nan, nan, nan, nan, 0.05555555555555565], dtype=object)}, 'REEGO': {'Adopters_6': array([nan, nan, nan, nan, nan, 0.09621993127147774], dtype=object), 'Followers_6': array([nan, nan, nan, nan, nan, 0.22164948453608246], dtype=object), 'Innovators_6': array([nan, nan, nan, nan, nan, 0.0034364261168384146], dtype=object), 'Early Adopters_5': array([nan, nan, nan, nan, nan, 0.0859106

#### Distances between all points

In [9]:
df_all_sem = son.get_comprehensive_df_semantic()
index = df_all_sem.index 

distances = an.compute_distance_centroids(df_all_sem, df_all_sem, max_distance_1D=6)[3]


df_out_sem = pd.DataFrame(columns=index, index=index)

for i, start in enumerate(index):
    index_to_search = index[i:]
    for stop in index_to_search:
        df_out_sem.loc[start, stop] = distances[start][stop]
        df_out_sem.loc[stop, start] = distances[start][stop]

df_out_sem

,MEMIMMO,REEGO,Adopters_6,Followers_6,Innovators_6
MEMIMMO,NaN,NaN,NaN,NaN,NaN
REEGO,NaN,NaN,NaN,NaN,NaN
Adopters_6,NaN,NaN,NaN,NaN,NaN
Followers_6,NaN,NaN,NaN,NaN,NaN
Innovators_6,NaN,NaN,NaN,NaN,NaN


## Multi Dimensional Scaling

In [10]:
son.df_segments_mds

,Segment,Period,Autonomy,Sophistication,Economy
Index,,,,,


In [11]:
son.df_brands_mds

,Autonomy,Sophistication,Economy
MARKET : Vodites,,,


### Compute Distances between Segments and Products 

#### Compute closest brands for each segment

In [12]:
df_seg_mds = son.df_segments_mds[:5][['Economy', 'Performance', 'Convenience']]

_ = an.get_n_closest(df_base=df_seg_mds, df_performers=son.df_brands_mds, num_top=3, max_distance_1D=40, weighted="eq")

KeyError: "['Performance', 'Convenience'] not in index"

#### Compute closest Segments for each Brand

In [ ]:
_ = an.get_n_closest(df_base=son.df_brands_mds, df_performers=df_seg_mds, num_top=3, max_distance_1D=40, weighted="eq")

---------- MOST ----------
Segment:	 Savers_6
Distance:	 0.9512096320981283
Segment:	 Shoppers_6
Distance:	 0.7622406679013419
Segment:	 High Earners_6
Distance:	 0.6048735434488515

---------- MOVE ----------
Segment:	 Explorers_6
Distance:	 0.9833391676878575
Segment:	 Professionals_6
Distance:	 0.7937071256683838
Segment:	 High Earners_6
Distance:	 0.7870158456598237

---------- ROADY ----------
Segment:	 Savers_6
Distance:	 0.9781940375126434
Segment:	 Shoppers_6
Distance:	 0.7773848612515313
Segment:	 Explorers_6
Distance:	 0.6071494134746223

---------- ROBUDO ----------
Segment:	 Shoppers_6
Distance:	 0.8972624703431117
Segment:	 Savers_6
Distance:	 0.8760678949316737
Segment:	 Explorers_6
Distance:	 0.7329249481263116

---------- ROCK ----------
Segment:	 Professionals_6
Distance:	 0.9458566716944737
Segment:	 High Earners_6
Distance:	 0.8450639917041017
Segment:	 Explorers_6
Distance:	 0.7477735805537677

---------- ROLLED ----------
Segment:	 High Earners_6
Distance:	 0.97793

#### Distances between all points

In [ ]:
df_all_mds = son.get_comprehensive_df_mds()
index = df_all_mds.index 

distances = an.compute_distance_centroids(df_all_mds, df_all_mds, weighted="eq", max_distance_1D=40)[3]


df_out_mds = pd.DataFrame(columns=index, index=index)

for i, start in enumerate(index):
    index_to_search = index[i:]
    for stop in index_to_search:
        df_out_mds.loc[start, stop] = distances[start][stop]
        df_out_mds.loc[stop, start] = distances[start][stop]

df_out_mds

,MOST,MOVE,ROADY,ROBUDO,ROCK,ROLLED,SOFT,SOLO,TONE,Explorers_6,High Earners_6,Professionals_6,Savers_6,Shoppers_6
MOST,1.0,0.600574,0.969308,0.853129,0.487333,0.588915,0.811341,0.788791,0.528741,0.598018,0.604874,0.48359,0.95121,0.762241
MOVE,0.600574,1.0,0.60868,0.736578,0.763188,0.792535,0.516804,0.707683,0.740202,0.983339,0.787016,0.793707,0.622686,0.775251
ROADY,0.969308,0.60868,1.0,0.860949,0.484625,0.582061,0.818349,0.782427,0.521799,0.607149,0.597022,0.482094,0.978194,0.777385
ROBUDO,0.853129,0.736578,0.860949,1.0,0.616739,0.701467,0.710973,0.806478,0.644807,0.732925,0.71265,0.614632,0.876068,0.897262
ROCK,0.487333,0.763188,0.484625,0.616739,1.0,0.866757,0.353509,0.573348,0.920415,0.747774,0.845064,0.945857,0.495933,0.644468
ROLLED,0.588915,0.792535,0.582061,0.701467,0.866757,1.0,0.462537,0.69396,0.912367,0.776455,0.977934,0.86761,0.589634,0.695389
SOFT,0.811341,0.516804,0.818349,0.710973,0.353509,0.462537,1.0,0.748275,0.388055,0.519752,0.479133,0.36285,0.806161,0.636138
SOLO,0.788791,0.707683,0.782427,0.806478,0.573348,0.69396,0.748275,1.0,0.610954,0.705675,0.711673,0.588413,0.779337,0.727342
TONE,0.528741,0.740202,0.521799,0.644807,0.920415,0.912367,0.388055,0.610954,1.0,0.72364,0.895633,0.888342,0.530557,0.65253
Explorers_6,0.598018,0.983339,0.607149,0.732925,0.747774,0.776455,0.519752,0.705675,0.72364,1.0,0.771315,0.779225,0.621364,0.773272


# Forecast analysis

In [ ]:
df_seg_sem_fc = an.forecast_df(son.df_segments_semantic, steps=2)

df_seg_sem_fc = df_seg_sem_fc[(df_seg_sem_fc["Period"]>an.last_period)].drop(labels=["Period", "Segment"], axis=1)
df_seg_sem_fc

,# Features,Design Index,Battery Life,Display Size,Proc. Power,Price
Explorers_8,4.07,3.5,4.9,5.48,5.35,5.41
Professionals_8,4.42,5.8,4.79,6.02,5.92,6.14
High Earners_8,3.29,6.24,3.3,3.73,4.29,5.33
Shoppers_8,2.39,4.46,2.8,5.2,4.85,2.45
Savers_8,2.87,4.41,2.13,3.52,2.7,2.05
High Earners_7,3.29,6.27,3.35,3.83,4.38,5.37
Professionals_7,4.52,5.81,4.84,5.94,5.88,6.0
Savers_7,2.82,4.29,2.06,3.4,2.6,2.06
Shoppers_7,2.29,4.53,2.81,5.04,4.75,2.55
Explorers_7,4.15,3.21,5.04,5.53,5.46,5.1


In [ ]:
df_all_sem = pd.concat([df_seg_sem_fc, son.df_brands_semantic], join="inner")
index = df_all_sem.index

distances = an.compute_distance_centroids(df_all_sem, df_all_sem, weighted="default", max_distance_1D=6)[3]


df_out_sem_fc = pd.DataFrame(columns=index, index=index)

for i, start in enumerate(index):
    index_to_search = index[i:]
    for stop in index_to_search:
        df_out_sem_fc.loc[start, stop] = distances[start][stop]
        df_out_sem_fc.loc[stop, start] = distances[start][stop]

df_out_sem_fc

,Explorers_8,Professionals_8,High Earners_8,Shoppers_8,Savers_8,High Earners_7,Professionals_7,Savers_7,Shoppers_7,Explorers_7,MOST,MOVE,ROADY,ROBUDO,ROCK,ROLLED,SOFT,SOLO,TONE
Explorers_8,1.0,0.830126,0.773458,0.662723,0.55143,0.778029,0.83707,0.546443,0.669253,0.961266,0.524687,0.916236,0.526976,0.647727,0.814443,0.78965,0.47318,0.677443,0.802855
Professionals_8,0.830126,1.0,0.779408,0.576855,0.457035,0.78916,0.983676,0.450093,0.583703,0.801753,0.431082,0.769101,0.430398,0.562188,0.923411,0.818412,0.342432,0.566399,0.863673
High Earners_8,0.773458,0.779408,1.0,0.662555,0.613118,0.989454,0.788039,0.608414,0.676222,0.752403,0.603923,0.721287,0.599192,0.709046,0.795285,0.95608,0.5009,0.729653,0.865058
Shoppers_8,0.662723,0.576855,0.662555,1.0,0.795121,0.660354,0.590081,0.784757,0.982872,0.684624,0.735493,0.68991,0.752283,0.863434,0.614934,0.656802,0.61816,0.670768,0.628551
Savers_8,0.55143,0.457035,0.613118,0.795121,1.0,0.605965,0.470335,0.986574,0.803106,0.566508,0.931075,0.568566,0.948422,0.883714,0.496278,0.586881,0.765121,0.720981,0.539785
High Earners_7,0.778029,0.78916,0.989454,0.660354,0.605965,1.0,0.797683,0.60106,0.673747,0.756268,0.595673,0.724609,0.591201,0.703192,0.80441,0.966394,0.492516,0.721645,0.872651
Professionals_7,0.83707,0.983676,0.788039,0.590081,0.470335,0.797683,1.0,0.463267,0.596917,0.810016,0.443777,0.778381,0.44324,0.574833,0.937178,0.826697,0.353851,0.575656,0.876211
Savers_7,0.546443,0.450093,0.608414,0.784757,0.986574,0.60106,0.463267,1.0,0.792895,0.561308,0.940619,0.563151,0.960155,0.876264,0.488269,0.581289,0.77664,0.724832,0.532891
Shoppers_7,0.669253,0.583703,0.676222,0.982872,0.803106,0.673747,0.596917,0.792895,1.0,0.689928,0.745574,0.693946,0.761486,0.877677,0.621366,0.669212,0.626384,0.68548,0.638078
Explorers_7,0.961266,0.801753,0.752403,0.684624,0.566508,0.756268,0.810016,0.561308,0.689928,1.0,0.536191,0.950047,0.540226,0.659709,0.794696,0.766357,0.488402,0.678483,0.78213


In [ ]:
df_seg_mds_fc = an.forecast_df(son.df_segments_mds, steps=2)

df_seg_mds_fc = df_seg_mds_fc[(df_seg_mds_fc["Period"]>an.last_period)][["Economy",	"Performance", "Convenience"]]
df_seg_mds_fc

,Economy,Performance,Convenience
Explorers_8,-9.42,9.4,-1.06
Professionals_8,-14.14,13.02,9.8
High Earners_8,-8.78,0.88,9.06
Shoppers_8,10.3,6.34,-0.46
Savers_8,12.98,-7.04,-1.32
High Earners_7,-9.08,1.48,9.26
Professionals_7,-13.26,12.66,9.96
Savers_7,12.92,-7.74,-2.02
Shoppers_7,9.64,5.56,-0.22
Explorers_7,-7.34,9.96,-2.18


In [ ]:
son.df_brands_mds

,Economy,Performance,Convenience
MARKET : Sonites,,,
MOST,11.84,-11.38,-1.40
MOVE,-5.44,10.22,-2.20
ROADY,12.68,-9.94,-2.72
ROBUDO,7.70,-2.22,0.18
ROCK,-10.44,12.04,13.32
ROLLED,-9.68,3.54,9.80
SOFT,11.50,-16.36,-13.48
SOLO,-2.20,-9.56,-5.10
TONE,-9.82,6.80,14.92


In [ ]:
df_all_mds = pd.concat([df_seg_mds_fc, son.df_brands_mds], join="inner")
index = df_all_mds.index

distances = an.compute_distance_centroids(df_all_mds, df_all_mds, weighted="eq", max_distance_1D=40)[3]


df_out_mds_fc = pd.DataFrame(columns=index, index=index)

for i, start in enumerate(index):
    index_to_search = index[i:]
    for stop in index_to_search:
        df_out_mds_fc.loc[start, stop] = distances[start][stop]
        df_out_mds_fc.loc[stop, start] = distances[start][stop]

df_out_mds_fc

,Explorers_8,Professionals_8,High Earners_8,Shoppers_8,Savers_8,High Earners_7,Professionals_7,Savers_7,Shoppers_7,Explorers_7,MOST,MOVE,ROADY,ROBUDO,ROCK,ROLLED,SOFT,SOLO,TONE
Explorers_8,1.0,0.821276,0.808833,0.71183,0.598933,0.81217,0.825111,0.593343,0.719103,0.964957,0.570875,0.939083,0.575441,0.700815,0.788461,0.821846,0.488572,0.701416,0.766244
Professionals_8,0.821276,1.0,0.808158,0.605453,0.487337,0.817959,0.986084,0.478997,0.612288,0.796323,0.460757,0.78228,0.459317,0.591296,0.924943,0.848781,0.344486,0.573183,0.868055
High Earners_8,0.808833,0.808158,1.0,0.682297,0.633721,0.989896,0.817626,0.62696,0.694728,0.790408,0.622259,0.783408,0.613679,0.726118,0.825926,0.958084,0.496564,0.728893,0.878836
Shoppers_8,0.71183,0.605453,0.682297,1.0,0.80265,0.679297,0.61714,0.792062,0.984851,0.7389,0.742911,0.764668,0.760291,0.870543,0.631296,0.673303,0.621888,0.70049,0.634404
Savers_8,0.598933,0.487337,0.633721,0.80265,1.0,0.626065,0.499195,0.985685,0.811185,0.617399,0.935222,0.635429,0.953318,0.894564,0.515474,0.604961,0.777833,0.771294,0.549278
High Earners_7,0.81217,0.817959,0.989896,0.679297,0.626065,1.0,0.827425,0.619144,0.691457,0.792931,0.613601,0.785442,0.605392,0.719485,0.835527,0.968065,0.487874,0.720333,0.887375
Professionals_7,0.825111,0.986084,0.817626,0.61714,0.499195,0.827425,1.0,0.490696,0.624042,0.801193,0.472233,0.788374,0.470766,0.603036,0.936056,0.858567,0.353753,0.58096,0.878572
Savers_7,0.593343,0.478997,0.62696,0.792062,0.985685,0.619144,0.490696,1.0,0.80058,0.611685,0.944471,0.629279,0.966498,0.885837,0.505814,0.597479,0.792007,0.775736,0.540045
Shoppers_7,0.719103,0.612288,0.694728,0.984851,0.811185,0.691457,0.624042,0.80058,1.0,0.745244,0.752851,0.770398,0.769176,0.884123,0.638139,0.684517,0.629254,0.714002,0.643674
Explorers_7,0.964957,0.796323,0.790408,0.7389,0.617399,0.792931,0.801193,0.611685,0.745244,1.0,0.585704,0.972319,0.592492,0.718589,0.769879,0.800933,0.505156,0.705616,0.746464


## MOST - Next Period

- Old Contribution: 102$ 

In [ ]:
from Utils import combined_error

target_sem = [2.8, 4.2, 2, 3.4, 2.6, 2.1]
target_mds = [12.9, -7.7, -2.2]

errors = []
contributions_array = []
prices_array = np.linspace(200, 250, num=8)

for price in prices_array:
    price = round(price)
    print(f"Price {price}")
    new_features = np.array([9, 7, 36, 16, 27, price])
    res = combined_error(features=new_features, ideal_semantic=target_sem, ideal_mds=target_mds, semantic_weights=son.rel_importance_features, mds_weights=[1/3, 1/3,1/3], error_weights=[1,1], model=solver)
    errors.append(res)
    print(f"Combined_error: {res}")
    contr = an.compute_contribution(price, 49, [11/42, 26/42, 5/42])
    contributions_array.append(contr)
    print(f"Contribution: {contr}")
    print()



Price 200
semantic_error 0.061320191581535455 mds_error 0.07003308842764255
Combined_error: 0.131353280009178
Contribution: 76

Price 207
semantic_error 0.052012334527434256 mds_error 0.05959356987325637
Combined_error: 0.11160590440069063
Contribution: 81

Price 214
semantic_error 0.04432793143496294 mds_error 0.05010788332941529
Combined_error: 0.09443581476437823
Contribution: 85

Price 221
semantic_error 0.03923281257472311 mds_error 0.04222384017662084
Combined_error: 0.08145665275134395
Contribution: 90

Price 229
semantic_error 0.03791862628902454 mds_error 0.036520475460056434
Combined_error: 0.07443910174908097
Contribution: 95

Price 236
semantic_error 0.04106262604110922 mds_error 0.03571770156820919
Combined_error: 0.07678032760931841
Contribution: 99

Price 243
semantic_error 0.04742385275416561 mds_error 0.039120449751934516
Combined_error: 0.08654430250610012
Contribution: 104

Price 250
semantic_error 0.055914844876983394 mds_error 0.04580077814036054
Combined_error: 0.

### Semantic Scales

In [ ]:
new_features = np.array([9, 7, 36, 16, 27, 240])

semantic_regressed = np.array(solver.regress_semantic(new_features))

semantic_regressed = [round(sem, 1) for sem in semantic_regressed]
semantic_regressed

[2.5, 4.3, 2.0, 3.0, 2.3, 2.3]

In [ ]:
res = an.compute_distance_centroids(semantic_regressed, target_sem)
res[3]

{'observation': {'centroid': 0.9584950437107272}}

In [ ]:
res[4]

{'observation': {'centroid': array([0.3, 0.1, 0. , 0.4, 0.3, 0.2])}}

### Multi Dimensional Scaling

In [ ]:
mds_regressed = np.array(solver.regress_mds(new_features))
mds_regressed 

array([ 12.05189471, -10.13009993,  -2.31694359])

In [ ]:
res = an.compute_distance_centroids(mds_regressed, target_mds, max_distance_1D=40, weighted="eq")
res[3]

{'observation': {'centroid': 0.9628114487219342}}

### Sensitivity Analysis

## MOVE - Next Period

In [ ]:
move_features = son.move_features.drop("Base Cost", axis=1).values.flatten()

In [ ]:
from Utils import combined_error

explorers_semantic_mk = [4.2, 3.2, 5.0, 5.5, 5.4, 5.1]
explorers_mds_mk = [-7.5, 9.7, -2.0]

for price in np.linspace(350, 350*1.2, num=8):
    price = round(price)
    print(f"Price {price}")
    new_features = np.array([13, 6, 82, 31, 83, price])
    res = combined_error(features=new_features, ideal_semantic=explorers_semantic_mk, ideal_mds=explorers_mds_mk, semantic_weights=son.rel_importance_features, mds_weights=[1/3, 1/3,1/3], error_weights=[1,1], model=solver)
    print(f"Combined_error: {res}")
    contr = an.compute_contribution(price, 107, [28/40, 5/40, 7/40])
    print(f"Contribution: {contr}")
    print()

Price 350
semantic_error 0.10352620886661656 mds_error 0.0795765630535199
Combined_error: 0.18310277192013646
Contribution: 109

Price 360
semantic_error 0.08759464463835653 mds_error 0.06224934553985306
Combined_error: 0.1498439901782096
Contribution: 115

Price 370
semantic_error 0.07259433900064505 mds_error 0.04533857450962653
Combined_error: 0.11793291351027158
Contribution: 121

Price 380
semantic_error 0.05923702203146375 mds_error 0.029567640127099115
Combined_error: 0.08880466215856286
Contribution: 127

Price 390
semantic_error 0.04888846430362315 mds_error 0.018192251990945985
Combined_error: 0.06708071629456913
Contribution: 134

Price 400
semantic_error 0.0437391077112016 mds_error 0.020633090122767928
Combined_error: 0.06437219783396952
Contribution: 140

Price 410
semantic_error 0.04558626529309584 mds_error 0.03403739809712547
Combined_error: 0.07962366339022131
Contribution: 146

Price 420
semantic_error 0.05371290405449958 mds_error 0.050293321784565914
Combined_error

In [ ]:
move_new_features = [13, 6, 82, 31, 83, 400]

semantic_move = solver.regress_semantic(move_new_features)
semantic_move = [round(sem,1) for sem in semantic_move]
semantic_move

[4.1, 3.4, 5.8, 5.6, 5.7, 5.1]

In [ ]:
explorers_semantic_mk

[4.2, 3.2, 5.0, 5.5, 5.4, 5.1]

In [ ]:
an.compute_distance_centroids(semantic_move, explorers_semantic_mk)[3]

{'observation': {'centroid': 0.9556682368253326}}

In [ ]:
an.compute_distance_centroids(semantic_move, explorers_semantic_mk)[4]

{'observation': {'centroid': array([0.1, 0.2, 0.8, 0.1, 0.3, 0. ])}}

In [ ]:
mds_move = solver.regress_mds(move_new_features)
mds_move = [round(mds) for mds in mds_move] 
mds_move

[-7, 10, -1]

In [ ]:
an.compute_distance_centroids(mds_move, explorers_mds_mk, weighted="eq", max_distance_1D=40)[3]

{'observation': {'centroid': 0.9832917186201972}}

In [ ]:
son.move_mds

,Economy,Performance,Convenience
MARKET : Sonites,,,
MOVE,-5.44,10.22,-2.2


In [ ]:
explorers_mds_mk

[-7.5, 9.7, -2.0]

In [ ]:
an.compute_distance_centroids(mds_move, explorers_mds_mk, weighted="eq", max_distance_1D=40)[4]

{'observation': {'centroid': array([0.5, 0.3, 1. ])}}